### Data Processing

#### The purpose of this notebook is to read in inintial run data, investigate features, and perform preprocessing for modeling.

In [7]:
from pathlib import Path

run_date = "02-05-26"
raw_data_path = Path(f"../data-raw/run_data_{run_date}.csv")
print(raw_data_path)

../data-raw/run_data_02-05-26.csv


In [8]:
from gait_analytics.preprocess import process_gait_data
from gait_analytics.visualization import plot_data

output_path = Path(f"../data-processed/{raw_data_path.stem}_preprocessed.parquet")
df_preprocessed = process_gait_data(raw_data_path, n_seconds_to_trim=10, output_path=output_path)
plot_data(df_preprocessed)

In [15]:
df_preprocessed.head()

timestamp,accel_x,accel_y,accel_z,speed_mps,accel_magnitude,is_step,cadence_steps_per_minute,accel_magnitude_smoothed,target_speed_mps_smoothed
f64,f64,f64,f64,f64,f64,bool,f64,f64,f64
1.7703e9,0.483978,0.834061,0.407425,1.68,1.046846,false,120.0,0.9962,1.4324
1.7703e9,0.243866,0.650787,0.193787,1.68,0.72149,false,120.0,0.872801,1.434
1.7703e9,-0.027634,0.473312,-0.230881,1.68,0.527346,false,120.0,0.892997,1.4356
1.7703e9,-0.028717,0.182037,-0.811218,1.68,0.831888,false,120.0,0.866732,1.4372
1.7703e9,0.272141,0.252701,-1.284821,1.68,1.337416,true,140.0,0.939534,1.4388


In [17]:
df_preprocessed.write_parquet("../data-processed/run_data_2-4-26_processed.parquet")